In [ ]:
#pip install nltk

In [144]:
# Import functions
from pyspark.sql.types import *
from pyspark.sql import functions 
from pyspark.sql import SparkSession 

from pyspark.sql.functions import col,udf,max as max_,min as min_,expr,to_date
from pyspark.sql.types import DecimalType

import pandas as pd
import numpy as np
import seaborn as sns

import string
from string import punctuation
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')


import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

[nltk_data] Downloading package stopwords to /home/hduser/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
#Read the csv file from hadoop and then create a temp view using spark sql

elontweetspath = "/user1/Elonmusktweets.csv"

# Obtain Tweets dataset
elontweets = spark.read.csv(elontweetspath, header='true')
elontweets.createOrReplaceTempView("elontweets")

# Cache the Tweets dataset 
elontweets.cache()
elontweets.first()

2023-05-14 11:45:44,095 WARN util.package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


Row(id='1282939902531790000', conversation_id='1282933079431150000', created_at='1594711683000', date='14/07/2020', time='07:28:03', timezone='UTC', user_id='44196397', username='elonmusk', name='Elon Musk', place=None, tweet='Cute', mentions="['teslarati']", urls='[]', photos='[]', replies_count='222', retweets_count='176', likes_count='7883', hashtags='[]', cashtags='[]', link='https://twitter.com/elonmusk/status/1282939902531796993', retweet='FALSE', quote_url=None, video='0', near=None, geo=None, source=None, user_rt_id=None, user_rt=None, retweet_id=None, reply_to="[{'user_id': '44196397', 'username': 'elonmusk'}, {'user_id': '1308211178', 'username': 'Teslarati'}]", retweet_date=None, translate=None, trans_src=None, trans_dest=None)

In [118]:
#Fetch the records and store in spark dataframe
df=spark.sql("select * from elontweets")
df


DataFrame[id: string, conversation_id: string, created_at: string, date: string, time: string, timezone: string, user_id: string, username: string, name: string, place: string, tweet: string, mentions: string, urls: string, photos: string, replies_count: string, retweets_count: string, likes_count: string, hashtags: string, cashtags: string, link: string, retweet: string, quote_url: string, video: string, near: string, geo: string, source: string, user_rt_id: string, user_rt: string, retweet_id: string, reply_to: string, retweet_date: string, translate: string, trans_src: string, trans_dest: string]

In [99]:
#Fetch the relevant columsn and store in spark dataframe

df_tweet=spark.sql("select  id,date,tweet from elontweets")
df_tweet.dtypes

[('id', 'string'), ('date', 'string'), ('tweet', 'string')]

In [21]:
#Define a function to remove the stop words from the tweet

def get_text_processing(text):
    stpword = stopwords.words('english')
    no_punctuation = [char for char in text if char not in string.punctuation]
    no_punctuation = ''.join(no_punctuation)
    return ' '.join([word for word in no_punctuation.split() if word.lower() not in stpword])

In [22]:
#Call the function to remove the stop words and text processing
get_text_processing_udf=udf(lambda x:get_text_processing(x))

In [78]:
#Select non null tweets
clean_tweet=df_tweet.select (col("id"),col('date'),get_text_processing_udf("tweet").alias("clean_tweet"))
                    .filter("tweet is not null")


In [37]:
clean_tweet.show()

+-------------------+----------+--------------------+
|                 id|      date|         clean_tweet|
+-------------------+----------+--------------------+
|1282939902531790000|14/07/2020|                Cute|
|1282844872571900000|14/07/2020|                 Wow|
|1282805559834490000|13/07/2020|Reusability essen...|
|1282800187308570000|13/07/2020|          Wild times|
|1282800078000800000|13/07/2020|We’re extra paran...|
|1282783693170300000|13/07/2020|     Welcome anytime|
|1282757235186130000|13/07/2020|Well care much su...|
|1282501634166120000|13/07/2020|Yes plan Supercha...|
|1282500394619300000|13/07/2020|                  👀|
|1282500077995520000|13/07/2020|               think|
|1282499968331210000|13/07/2020|range testing Num...|
|1282499399562580000|13/07/2020|reduced pricing M...|
|1282498919025390000|13/07/2020|                 Yes|
|1282497172982780000|13/07/2020|may able reach 25...|
|1282496094908240000|13/07/2020|range would unacc...|
|1282495504350230000|13/07/20

In [127]:
clean_tweet.select (clean_tweet.id.cast(StringType())).show()

+-------------------+
|                 id|
+-------------------+
|1282939902531790000|
|1282844872571900000|
|1282805559834490000|
|1282800187308570000|
|1282800078000800000|
|1282783693170300000|
|1282757235186130000|
|1282501634166120000|
|1282500394619300000|
|1282500077995520000|
|1282499968331210000|
|1282499399562580000|
|1282498919025390000|
|1282497172982780000|
|1282496094908240000|
|1282495504350230000|
|1282492573383180000|
|1282450019413420000|
|1282449465324920000|
|1282185119734700000|
+-------------------+
only showing top 20 rows



In [160]:
clean_tweet_date=clean_tweet.select(to_date(clean_tweet.date,'dd/MM/yyyy').alias('dt'))

In [162]:
clean_tweet_date.select (max('dt')).show()

+----------+
|   max(dt)|
+----------+
|2020-07-14|
+----------+



In [163]:
clean_tweet_date.select (min_('dt')).show()

+----------+
|   min(dt)|
+----------+
|2015-01-30|
+----------+



In [59]:
import nltk


In [60]:
nltk.download([
            "names",
            "stopwords",
            "averaged_perceptron_tagger",
            "vader_lexicon",
            "punkt",
        ])

[nltk_data] Downloading package names to /home/hduser/nltk_data...
[nltk_data]   Package names is already up-to-date!
[nltk_data] Downloading package stopwords to /home/hduser/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/hduser/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/hduser/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to /home/hduser/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [61]:
#Use NLTK library for Sentimentintensity analyser
from nltk.sentiment import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()


In [62]:
from random import shuffle

#Define a function to generate sentiments
def is_positive(tweet: str) -> bool:
    """True if tweet has positive compound sentiment, False otherwise."""
    if sia.polarity_scores(tweet)["compound"] > 0:
        return 1
    elif sia.polarity_scores(tweet)["compound"] < 0:
        return 0
    else: return 2
    



In [63]:
#Create a RDD mapping for tweets and its sentiments
rdd=clean_tweet.rdd.map(lambda x: (x["id"],x["date"],x["clean_tweet"],is_positive(x["clean_tweet"])))

In [64]:
#Next step is to collect the rdd mapping and assign to a data variable
data=rdd.collect()

In [65]:
print(data)

[('1282939902531790000', '14/07/2020', 'Cute', 1), ('1282844872571900000', '14/07/2020', 'Wow', 1), ('1282805559834490000', '13/07/2020', 'Reusability essential rocket single use absurd single use airplane F9 engines already fire 3 times per flight', 0), ('1282800187308570000', '13/07/2020', 'Wild times', 2), ('1282800078000800000', '13/07/2020', 'We’re extra paranoid Maximizing probability successful launch paramount', 1), ('1282783693170300000', '13/07/2020', 'Welcome anytime', 1), ('1282757235186130000', '13/07/2020', 'Well care much sustainability civilization truth irony part haha', 1), ('1282501634166120000', '13/07/2020', 'Yes plan Superchargers public high power wall connectors keep growing exponentially every year', 1), ('1282500394619300000', '13/07/2020', '👀', 2), ('1282500077995520000', '13/07/2020', 'think', 2), ('1282499968331210000', '13/07/2020', 'range testing Number significantly higher 300 Extremely good EV especially SUV', 1), ('1282499399562580000', '13/07/2020', '

In [66]:
df_clean_tweet=pd.DataFrame(data)

In [67]:
df_clean_tweet

,0,1,2,3
0,1282939902531790000,14/07/2020,Cute,1
1,1282844872571900000,14/07/2020,Wow,1
2,1282805559834490000,13/07/2020,Reusability essential rocket single use absurd...,0
3,1282800187308570000,13/07/2020,Wild times,2
4,1282800078000800000,13/07/2020,We’re extra paranoid Maximizing probability su...,1
...,...,...,...,...
9374,564563818022313000,08/02/2015,Prob good though give us time replace 1st stag...,1
9375,564562686478807000,08/02/2015,Air Force tracking radar went Launch postponed...,0
9376,564509965612634000,08/02/2015,Rocket reentry much tougher time around due de...,1
9377,564493608351313000,08/02/2015,Launching 1st deep space mission today Headed ...,2


In [68]:
df_clean_tweet.rename(columns= {0:'tweetid' , 1:'Date',2:'Tweet',3:'sentiment'} ,inplace=True)

In [69]:
df_clean_tweet.reset_index(drop=True,inplace=True)

In [70]:
#pip show pandas

In [71]:
#pip install pandas==1.2.5 (2.0.0)

In [80]:
df_clean_tweet.shape

(9379, 4)

In [166]:
#Export the file to hadoop filesystem
#clean_tweet.write.mode("append").csv("/user1")